In [ ]:
## Import dependent libraries
import os
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from keras.models import Model
from keras.layers import Input, Concatenate, Reshape
from keras import optimizers, callbacks
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.initializers import RandomNormal
## import required HIRNN
from libs.HIRNN_UP import HIRNNLayer, ConvLayer
from libs import hirnnutils
from keras import initializers, constraints, regularizers
from keras.layers import Layer, Dense, Lambda, Activation
import keras.backend as K
import tensorflow as tf
from keras.layers import Dropout, Activation
from keras.regularizers import l2
from tensorflow.keras.layers import concatenate 
from tensorflow.keras.backend import zeros 

## Ignore all the warnings
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['KMP_WARNINGS'] = '0'

In [ ]:
# import time period details
stn_data = pd.read_csv('/HIRNN_codes/sample_input_data_HIRNN/unmanaged_intermittent_catchments_time_period_details.csv')
stn_data

In [ ]:
# train-test split
def generate_train_test(train_set, test_set, wrap_length):
    train_x_np = train_set.values[:, :-1]
    train_y_np = train_set.values[:, -1:]
    test_x_np = test_set.values[:, :-1]
    test_y_np = test_set.values[:, -1:]
    
    wrap_number_train = (train_set.shape[0]-wrap_length)//365 + 1
    
    train_x = np.empty(shape = (wrap_number_train, wrap_length, train_x_np.shape[1]))
    train_y = np.empty(shape = (wrap_number_train, wrap_length, train_y_np.shape[1]))

    test_x = np.expand_dims(test_x_np, axis=0)
    test_y = np.expand_dims(test_y_np, axis=0)
    
    for i in range(wrap_number_train):
        train_x[i, :, :] = train_x_np[i*365:(wrap_length+i*365), :]
        train_y[i, :, :] = train_y_np[i*365:(wrap_length+i*365), :]
             
    return train_x, train_y, test_x, test_y



In [ ]:
def create_model(input_shape, seed, num_filters = None, model_type='physical', kernel_size = None, dropout_rate = None):
    """Create a Keras model with regularization and dropout.
    """
    
    # Set seeds for reproducibility
    tf.random.set_seed(seed)
    np.random.seed(seed)
    
    # Define input layer
    x_input = Input(shape=input_shape, name='Input')
    
    if model_type == 'physical':
        hydro_output = HIRNNLayer(mode= 'normal', name='Hydro')(x_input)
        model = Model(x_input, hydro_output)
    
    elif model_type == 'hybrid':
        cnn_output = ConvLayer(filters=num_filters, kernel_size=5, padding='causal', seed=seed, name='Conv1')(x_input)
        cnn_output = ConvLayer(filters=1, kernel_size=1, padding='causal', seed=seed, name='Conv2')(cnn_output)
        model = Model(x_input, cnn_output)
    
    return model

def train_model(model, train_x, train_y, ep_number, lrate, save_path, seed):
    """Train a Keras model.
    -- model: the Keras model object
    -- train_x, train_y: the input and target for training the model
    -- ep_number: the maximum epoch number
    -- lrate: the initial learning rate
    -- save_path: where the model will be saved
    """
     # Set seeds for reproducibility
    tf.random.set_seed(seed)
    np.random.seed(seed)
    
    save = callbacks.ModelCheckpoint(save_path, verbose=0, save_best_only=True, monitor='nse_metrics', mode='max',
                                     save_weights_only=True)
    es = callbacks.EarlyStopping(monitor='nse_metrics', mode='max', verbose=1, patience=20, min_delta=0.0005,
                                 restore_best_weights=True)
    reduce = callbacks.ReduceLROnPlateau(monitor='nse_metrics', factor=0.8, patience=5, verbose=1, mode='max',
                                         min_delta=0.0005, cooldown=0, min_lr=lrate / 100)
    tnan = callbacks.TerminateOnNaN()

    model.compile(loss=hirnnutils.nse_loss, metrics=[hirnnutils.nse_metrics], optimizer=optimizers.Adam(learning_rate=lrate))

    # Use the shuffled data for training
    history = model.fit(train_x, train_y, epochs=ep_number, batch_size=10000, callbacks=[save, es, reduce, tnan], shuffle=False, verbose=0)
        
    return history

def test_model(model, test_x, save_path):
    """Test a Keras model.
    -- model: the Keras model object
    -- test_x: the input for testing the model
    -- save_path: where the model was saved
    """
    model.load_weights(save_path)  
    pred_y = model.predict(test_x, batch_size=10000)
    return pred_y    


In [ ]:
# performance metrics
def NS(s,o):
    
    #Nash Sutcliffe efficiency coefficient
    #input:
        #s: simulated
        #o: observed
    
    return 1 - sum((s-o)**2)/sum((o-np.mean(o))**2)

def pc_bias(s,o):

#     Percent Bias
#     input:
#         s: simulated
#         o: observed

    return 100.0*sum(o-s)/sum(o)
def rmse(s,o):

#     Root Mean Squared Error
#     input:
#         s: simulated
#         o: observed

    return np.sqrt(np.mean((s-o)**2))


def kge(s,o):
    
#     Kling Gupta Efficiency 
#     input:
#         s: simulated
#         o: observed
    
    alpha = np.std(s)/np.std(o)
    beta = np.mean(s)/np.mean(o)
    return 1-((1 - np.corrcoef(s,o)[0,1])**2 + (alpha - 1)**2 + (beta - 1)**2)**0.5


In [ ]:
perform = pd.DataFrame()
perform_list = []
perform[['Station']] = ""
perform[['NSE_val', 'PBIAS_val', 'RMSE_val', 'KGE_val']] = ""

for stn in range(0, len(stn_data)): 
    station = stn_data['gauge_id'][stn]
    cat_file = str(station) + str('_input_data.csv')
    hirnn_data = pd.read_csv('/HIRNN_codes/sample_input_data_HIRNN/'+ cat_file)
    hirnn_data['Date'] = pd.to_datetime(hirnn_data['Date'])

    # Keep only the specified columns
    hirnn_data = hirnn_data[['Date', 'Pptn', 'PET', 'streamflow_mm']]
    hirnn_data.set_index('Date', inplace=True)
    PRNN_cal_start = stn_data['warm_up_cal_start'][stn]
    cal_end = stn_data['cal_end'][stn] 
    PRNN_val_start = stn_data['warm_up_val_start'][stn] 
    val_end = stn_data['val_end'][stn] 
    val_start = stn_data['val_start'][stn] 
    ####################
    #  Period set up   #
    ####################

    training_start = PRNN_cal_start
    training_end= cal_end

    testing_start = PRNN_val_start 
    testing_end= val_end

    # Split data set to training_set and testing_set
    train_set = hirnn_data[hirnn_data.index.isin(pd.date_range(training_start, training_end))]
    test_set = hirnn_data[hirnn_data.index.isin(pd.date_range(testing_start, testing_end))]

    print(f"The training data set is from {training_start} to {training_end}, with a shape of {train_set.shape}")
    print(f"The testing data set is from {testing_start} to {testing_end}, with a shape of {test_set.shape}")
    wrap_length= 2922 # It can be other values, but recommend this value should not be less than 5 years (1826 days).
    train_x, train_y, test_x, test_y = generate_train_test(train_set, test_set, wrap_length=wrap_length)

    print(f'The shape of train_x, train_y, test_x, and test_y after wrapping by {wrap_length} days are:')
    print(f'{train_x.shape}, {train_y.shape}, {test_x.shape}, and {test_y.shape}')
   


    basin_id =  str(station) + str('_HIRNN_UP') 
    save_path_hirnn = f'/HIRNN_codes/{basin_id}_physical.weights.h5' # use desired path


    model = create_model((train_x.shape[1], train_x.shape[2]), seed = 200, num_filters = 8, model_type='physical')
    model.summary()
    hybrid_history = train_model(model, train_x, train_y, ep_number=200, seed = 200, lrate=0.01, save_path=save_path_hirnn)
    ####################
    # Physical NN model#
    ####################
    model = create_model((test_x.shape[1], test_x.shape[2]), seed = 200, num_filters = 8, model_type='physical')
    Q_hirnn = test_model(model, test_x, save_path_hirnn)
    Q_hirnn = test_model(model, test_x, save_path_hirnn)
    evaluate_set = test_set.loc[:, ['Pptn', 'PET', 'streamflow_mm']]
    evaluate_set['Q_obs'] = evaluate_set['streamflow_mm']
    evaluate_set['Q_hirnn'] = np.clip(Q_hirnn[0, :, :], a_min = 0, a_max = None)
    evaluate_set = evaluate_set.loc[val_start:val_end]
    evaluate_set = evaluate_set.fillna(0)
    NSE_val = NS(evaluate_set['Q_hirnn'],evaluate_set['Q_obs'])
    PBIAS_val = pc_bias(evaluate_set['Q_hirnn'],evaluate_set['Q_obs'])
    RMSE_val = rmse(evaluate_set['Q_hirnn'],evaluate_set['Q_obs'])
    KGE_val = kge(evaluate_set['Q_hirnn'],evaluate_set['Q_obs'])    
    perform_data = {
        'Station': station,
        'NSE_val': NSE_val,
        'PBIAS_val': PBIAS_val,
        'RMSE_val': RMSE_val,
        'KGE_val': KGE_val
    }
    print(perform_data)
    perform_df = pd.DataFrame(perform_data, index=[0])
    perform_list.append(perform_df)

# Concatenate all dataframes in the list into one dataframe
perform = pd.concat(perform_list, ignore_index=True)    

In [ ]:
perform